# Trading EDA

To install conda and binance packages to this notebook uncomment the code below

In [1]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
%pip install binance-connector==1.13.0
%pip install pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Init binance client

In [2]:
from binance.spot import Spot as Client
import os
import sys
import yaml

# Load 
def load_config():
    """
    Load config from cfg folder respecting the order: defaults, app.yaml, environment vars
    """
    # Defaults
    cfg_dir="./../biml/cfg"
    default_cfg_path = f"{cfg_dir}/app-defaults.yaml"
    with open(default_cfg_path, "r") as appdefaults:
        config = yaml.safe_load(appdefaults)

    # Custom config, should contain custom information,
    cfg_path = f"{cfg_dir}/app.yaml"
    if os.path.exists(cfg_path):
        with open(cfg_path) as app:
            config.update(yaml.safe_load(app))
    else:
        sys.exit(
            f"Config {cfg_path} not found. Please copy cfg/app-defaults.yaml to {cfg_path} "
            f"and update connection info there.")

    # Enviroment variabless
    config.update(os.environ)
    return config


# Create binance client
config=load_config()
key,secret,url=config["biml.connector.key"],config["biml.connector.secret"],config["biml.connector.url"]
client = Client(key=key, secret=secret, base_url=url, timeout=10)


Get account info

In [3]:
account=client.account()
print(account)


{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'brokered': False, 'updateTime': 1664137479985, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00500000', 'locked': '0.00100000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '9885.14828000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}


Get orders from binance

In [13]:
# Get orders
import pandas as pd
import numpy as np
symbol="BTCUSDT"

# Get orders, but market orders have price=0
orders=pd.DataFrame(client.get_orders(symbol))
if not orders.empty: orders=orders.astype({'time': 'datetime64[ms]','updateTime':'datetime64[ms]','price':'float64','origQty':'float64'}).set_index('time',drop=False)

# Get trades and pull orders' prices from there
trades=pd.DataFrame( client.my_trades(symbol))
if not trades.empty: trades=trades.astype({'time': 'datetime64[ms]', 'price':'float64'}).set_index('time',drop=False)    
orders=orders.drop("price",axis=1).merge(trades[["orderId","price"]],on="orderId")

# Calculate profit of closing orders
orders["delta_price"]=orders["price"].diff()
orders["profit"] = orders["delta_price"]*orders["origQty"]
orders.loc[~orders["type"].isin(["LIMIT_MAKER", "STOP_LOSS_LIMIT"]),["profit"]]=np.nan

orders["time"] = orders["updateTime"]
orders.set_index("time", drop=False, inplace=True)
orders.tail()


,symbol,orderId,orderListId,clientOrderId,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,origQuoteOrderQty,price,delta_price,profit
time,,,,,,,,,,,,,,,,,,,,
2022-09-25 19:44:26.321,BTCUSDT,7364400,-1,HvmWIkpkh4a2AaoA5M6XCv,0.001,0.00100000,18.89678000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-09-25 19:44:26.321,2022-09-25 19:44:26.321,True,0.00000000,18896.78,-4.05,NaN
2022-09-25 19:46:40.144,BTCUSDT,7364405,14924,LItZNO02NJHTwlfnZYZbTK,0.001,0.00100000,18.87690000,FILLED,GTC,STOP_LOSS_LIMIT,SELL,18876.90000000,0.00000000,2022-09-25 19:46:40.144,2022-09-25 19:46:40.144,True,0.00000000,18876.90,-19.88,-0.01988
2022-09-25 19:50:47.153,BTCUSDT,7366225,-1,TUP3vNhMzN0IDf6vepgfgx,0.001,0.00100000,18.89208000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-09-25 19:50:47.153,2022-09-25 19:50:47.153,True,0.00000000,18892.08,15.18,NaN
2022-09-25 20:17:18.824,BTCUSDT,7366228,14925,miWBnZNSlos9Tf3ttkHKaf,0.001,0.00100000,18.86727000,FILLED,GTC,STOP_LOSS_LIMIT,SELL,18867.27000000,0.00000000,2022-09-25 20:17:18.824,2022-09-25 20:17:18.824,True,0.00000000,18867.27,-24.81,-0.02481
2022-09-25 20:24:39.191,BTCUSDT,7376054,-1,ax6njL61iPB3cNEBlbsIaR,0.001,0.00100000,18.89527000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-09-25 20:24:39.191,2022-09-25 20:24:39.191,True,0.00000000,18895.27,28.00,NaN


Look at opened orders

In [5]:
orders_opened=client.get_open_orders(symbol)
print(orders_opened)

[{'symbol': 'BTCUSDT', 'orderId': 7376058, 'orderListId': 14936, 'clientOrderId': 'bj7L66TGMwpbVeyscRYis0', 'price': '18867.93000000', 'origQty': '0.00100000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'STOP_LOSS_LIMIT', 'side': 'SELL', 'stopPrice': '18867.93000000', 'icebergQty': '0.00000000', 'time': 1664137479985, 'updateTime': 1664137479985, 'isWorking': False, 'origQuoteOrderQty': '0.00000000'}, {'symbol': 'BTCUSDT', 'orderId': 7376059, 'orderListId': 14936, 'clientOrderId': 'rBFRYRe4j5w3fZkhwM0I2a', 'price': '19377.01000000', 'origQty': '0.00100000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT_MAKER', 'side': 'SELL', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': 1664137479985, 'updateTime': 1664137479985, 'isWorking': True, 'origQuoteOrderQty': '0.00000000'}]


Get candles from local history starting from earliest trade date ending by latest trade date.

In [6]:
import os
import glob
import pandas as pd
from datetime import timedelta,date


def read_candles(start_date_inc, end_date_inc):
    """ Read days from start to end from local history"""
    symbol="BTCUSDT"    
    data_dir=f"./../data/{symbol}"
    print(f"start date:{start_date_inc}, end date:{end_date_inc}, all inclusive")
    candles = pd.DataFrame()
    cur_candles_list=[]
    for n in range(int ((end_date_inc - start_date_inc).days)+1):
        cur_date = start_date_inc + timedelta(n)    
        csv_path=f"{data_dir}/{cur_date}_{symbol}_1m.csv"     
        if not os.path.exists(csv_path): continue
        #print(f"Read {cur_date} candles from {csv_path}")
        cur_candles=pd.read_csv(csv_path,index_col="close_time")[["open","high","low","close","vol"]]
        cur_candles_list.append(cur_candles)
    candles = pd.concat(cur_candles_list)
    return candles

def clean_candles(df):
    """ Remove outliers """
    q_low = df["low"].quantile(0.01)
    q_high  = df["high"].quantile(0.99)
    df=df[df["high"]< q_high]
    df=df[df["low"] > q_low ]
    return df

# Read candles for last 3 days
last_date=orders.index.max().date()
first_date=last_date - timedelta(days=0)

candles = read_candles(first_date, last_date)
candles = clean_candles(candles)
candles.tail()


start date:2022-09-25, end date:2022-09-25, all inclusive


,open,high,low,close,vol
close_time,,,,,
2022-09-25 20:36:59.999,18945.82,18946.45,18945.82,18946.45,0.100000
2022-09-25 20:37:59.999,18947.53,18947.67,18942.57,18942.57,4.020076
2022-09-25 20:38:59.999,18942.76,18942.76,18942.76,18942.76,0.082882
2022-09-25 20:39:59.999,18941.31,18943.77,18941.31,18943.57,4.006930
2022-09-25 20:40:59.999,18943.03,18943.10,18943.03,18943.10,0.027394


## EDA

### Look at trades quantity cumulative sum

## Look at trades on candlestick chart

In [14]:
# Install a conda package in the current Jupyter kernel
#%conda install -c plotly plotly=5.9.0
# Draw candlechart
import plotly as py
from plotly import graph_objects as go
import pandas as pd

df=candles.copy()
#df=clean_candles(df)
fig = go.Figure(data=[ \
                    go.Candlestick(\
                        x=df.index,\
                        open=df['open'],\
                        high=df['high'],\
                        low=df['low'],\
                        close=df['close'])
                     ])

# Customize ay to distribute captions by y so they do not overlap by x
ay,aymax,aystep=0,50,10
# Annotate buy, sell on the chart
for traderec in orders[(orders["status"] == "FILLED") & (orders.index > df.index.min())].iterrows():
    ay=(ay+aystep)%aymax
    trade=traderec[1]
    color='green' if trade['side']=="BUY" else 'red'
    text_short='buy' if trade['side']=="BUY"  else 'sell'
    status= f", profit: {round(trade['profit'], 10)}" if ~np.isnan(trade['profit']) else ""
    #text=f"{trade['time']} {text_short} at {str(trade['price'])}, profit: {str(trade['profit'])}"
    text=f"{trade['time']} {text_short} at {str(trade['price'])} {status}"
    fig.add_annotation(x=trade['time'], \
                       y=trade['price'],\
                       text=text_short,\
                       hovertext=text,\
                       showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,
                       font={'color':color}
                      )
fig.show()


## Profit

In [8]:
#import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=orders[(orders.status=="FILLED")&(orders["profit"])].copy()
profit["cum_profit"] = profit["profit"].cumsum()

#Plot
fig = px.line(profit, x="time", y="profit")
fig.show()

In [9]:
import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=orders[(orders.status=="FILLED")&(orders["profit"])].copy()
#profit["profit"]=profit["price"].diff()*(profit["qty"]/profit["qty"])
profit["cum_profit"] = profit["profit"].cumsum()

#Plot
fig = px.line(profit, y="cum_profit")
fig.show()

## Predicted and actual low/high

In [10]:
import os
import glob
import pandas as pd
from datetime import timedelta,date
import matplotlib.pyplot as plt
import plotly.express as px

def read_predictions(start_date_inc, end_date_inc):
    """ Read days from start to end from local predictions history"""
    symbol="BTCUSDT"    
    data_dir=f"./../model-dev/PredictLowHighStrategy/Xy"
    print(f"start date:{start_date_inc}, end date:{end_date_inc}, all inclusive")
    df = pd.DataFrame()
    candles_list=[]
    for n in range(int ((end_date_inc - start_date_inc).days)+1):
        cur_date = start_date_inc + timedelta(n)    
        candles_path=f"{data_dir}/{cur_date}_{symbol}_candles.csv"     
        if not os.path.exists(candles_path): continue
        #print(f"Read {cur_date} candles from {csv_path}")
        cur_candles=pd.read_csv(candles_path,index_col="close_time")
        candles_list.append(cur_candles)
    df = pd.concat(candles_list)
    return df


# Read candles for last 3 days
last_date=orders.index.max().date()
first_date=last_date - timedelta(days=0)

df = read_predictions(first_date, last_date)
px.line(df[["low","high","fut_low","fut_high"]], markers=True)


start date:2022-09-25, end date:2022-09-25, all inclusive


## Look at the account

In [11]:
print(account)

{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'brokered': False, 'updateTime': 1664137479985, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00500000', 'locked': '0.00100000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '9885.14828000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}
